In [74]:
import requests
from bs4 import BeautifulSoup
import html5lib
import urllib.request, json

In [81]:
username = 'm-kosik'

repos = requests.get('https://github.com/' + username + '?tab=repositories') 
soup = BeautifulSoup(repos.content,'html5lib')

repository_list = []
number_of_stars = {}
total_stars = 0

for repo in soup.find_all(class_='wb-break-all'):
    repository_list.append(repo.find('a')['href'])
    repo_name = repo.find('a').text.strip(' \n')
    number_of_stars[repo_name] = None

for element in soup.find_all('svg', {'aria-label': 'star'}):
    repo_name = element.parent['href'].split('/')[-2]
    stars = int(element.parent.text.strip('\n '))
    number_of_stars[repo_name] = stars
    total_stars += stars

for k,v in number_of_stars.items():
    print(f'{k} : {v}')

print('Total stars: ' + str(total_stars))

# test idea - total stars equal to the sum number_of_stars.values()

house_price_prediction : None
langtons-ant : None
maze_solving : None
square-nim-game-analysis : None
language-localizer : None
ml-kaggle-challenges : None
Total stars: 0


In [86]:
def find_used_languages_by_percent(soup):
    languages_dict = {}

    header = soup.find(lambda elm: elm.name == "h2" and "Languages" in elm.text)
    child = header.find_next_siblings()[0].find('span')
    for element in child.find_all('span'):
        language = element['aria-label'].rsplit(' ', 1)[0]
        percent_usage = float(element['aria-label'].rsplit(' ', 1)[1])
        languages_dict[language] = percent_usage
    return languages_dict

total_language_use_in_bytes = {}
for repo in repository_list:
    try:
        with urllib.request.urlopen('https://api.github.com/repos' + repo) as url:
            data = json.loads(url.read().decode())
            repo_size = float(data['size'])
    except urllib.error.HTTPError as exception:
        repo_size = 1000
        print(exception)
        print('Languages found in all repositories will be printed alphabetically.')

    temp_repo = requests.get('https://github.com/' + repo) 
    soup = BeautifulSoup(temp_repo.content,'html5lib')
    try:
        languages_in_percent = find_used_languages_by_percent(soup)
    except AttributeError:
        print('There are no languages specified for this repository.')
    
    print(languages_in_percent)

    languages_in_bytes = {k: v*repo_size for k, v in languages_in_percent.items()}
    print(languages_in_bytes)

    for language, size in languages_in_bytes.items():
        try:
            total_language_use_in_bytes[language] += size
        except KeyError:
            total_language_use_in_bytes[language] = size

print(total_language_use_in_bytes)
        
    

HTTP Error 403: rate limit exceeded
Languages found in all repositories will be printed alphabetically.
{'Jupyter Notebook': 100.0}
{'Jupyter Notebook': 100000.0}
HTTP Error 403: rate limit exceeded
Languages found in all repositories will be printed alphabetically.
{'Jupyter Notebook': 100.0}
{'Jupyter Notebook': 100000.0}
HTTP Error 403: rate limit exceeded
Languages found in all repositories will be printed alphabetically.
{'Jupyter Notebook': 96.8, 'Python': 3.2}
{'Jupyter Notebook': 96800.0, 'Python': 3200.0}
HTTP Error 403: rate limit exceeded
Languages found in all repositories will be printed alphabetically.
{'Jupyter Notebook': 97.0, 'Python': 3.0}
{'Jupyter Notebook': 97000.0, 'Python': 3000.0}
HTTP Error 403: rate limit exceeded
Languages found in all repositories will be printed alphabetically.
There are no languages specified for this repository.
{'Jupyter Notebook': 97.0, 'Python': 3.0}
{'Jupyter Notebook': 97000.0, 'Python': 3000.0}
HTTP Error 403: rate limit exceeded
La

{'Jupyter Notebook': 590600.0, 'Python': 9400.0}
